# GRU model

## Setup

In [5]:
# !pip install gensim

In [89]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Masking, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import gensim.downloader as api

from nltk.corpus import stopwords

In [55]:
X = pd.read_csv("../processed_data/features_1000sample_400min_600cutoff.csv")
y = pd.read_csv("../processed_data/target_1000sample_400min_600cutoff.csv")

In [56]:
codes = {p: i for i, p in enumerate(y["party"].unique())}
# y = y["party"].map(codes)
y = OneHotEncoder(sparse_output=False).fit_transform(y["party"].values.reshape(-1, 1))
X = X["text"]

In [57]:
X.shape, y.shape

((7000,), (7000, 7))

## Preprocessing

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Embed the training and test sentences

In [99]:
stop_words = set(stopwords.words('english'))

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word not in stop_words and word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence.split())
        embed.append(embedded_sentence)
        
    return embed

In [12]:
word2vec_model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [83]:
X_train_embed = embedding(word2vec_model, X_train)
X_test_embed = embedding(word2vec_model, X_test)

### Pad sequences to ensure uniform input size

In [101]:
maxlen = max(max([len(x) for x in X_train_embed]), max([len(x) for x in X_test_embed]))  # Maximum sequence length
maxlen, [len(x) for x in X_train_embed[:10]]

(351, [244, 273, 299, 285, 238, 290, 303, 288, 278, 259])

In [102]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=maxlen)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=maxlen)

### Tokenize the text data

In [8]:
# tokenizer = Tokenizer(num_words=10000)  # Limit vocabulary size
# tokenizer.fit_on_texts(X_train)
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

## The model

In [24]:
# # Convert words to Word2Vec embeddings
# word_index = tokenizer.word_index
# embedding_matrix = np.zeros((len(word_index) + 1, 300))  # Assuming Word2Vec vectors are 300-dimensional

# for word, i in word_index.items():
#     if word in word2vec_model:
#         embedding_matrix[i] = word2vec_model[word]

# # Define the model
# embedding_layer = tf.keras.layers.Embedding(len(word_index) + 1,
#                                             300,  # Assuming Word2Vec vectors are 300-dimensional
#                                             weights=[embedding_matrix],
#                                             input_length=maxlen,
#                                             trainable=False)  # Freeze the embedding layer

In [103]:
# Define the model
model = Sequential()
model.add(Masking())
model.add(GRU(150, activation="tanh", return_sequences=True, recurrent_dropout=0.4, dropout=0.2,
             kernel_regularizer=regularizers.l2(0.01)))
model.add(GRU(150, recurrent_dropout=0.4, dropout=0.2, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(75, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(7, activation="softmax"))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(patience=3, restore_best_weights=True)

In [ ]:
# Train the model
model.fit(np.array(X_train_pad), np.array(y_train), epochs=100, batch_size=32, 
          validation_split=0.2, callbacks=[es], shuffle=True)

Epoch 1/100
140/140 [==============================] - 157s 1s/step - loss: 1.9388 - accuracy: 0.1710 - val_loss: 1.8977 - val_accuracy: 0.2000
Epoch 2/100
140/140 [==============================] - 156s 1s/step - loss: 1.8913 - accuracy: 0.2085 - val_loss: 1.8596 - val_accuracy: 0.2214
Epoch 3/100
140/140 [==============================] - 151s 1s/step - loss: 1.7523 - accuracy: 0.2692 - val_loss: 1.5835 - val_accuracy: 0.3205
Epoch 4/100
140/140 [==============================] - 152s 1s/step - loss: 1.5937 - accuracy: 0.3270 - val_loss: 1.5467 - val_accuracy: 0.3134
Epoch 5/100
140/140 [==============================] - 151s 1s/step - loss: 1.4702 - accuracy: 0.3737 - val_loss: 1.3686 - val_accuracy: 0.4384
Epoch 6/100
140/140 [==============================] - 152s 1s/step - loss: 1.3395 - accuracy: 0.4478 - val_loss: 1.3765 - val_accuracy: 0.4304
Epoch 7/100
140/140 [==============================] - 151s 1s/step - loss: 1.2865 - accuracy: 0.4725 - val_loss: 1.2499 - val_accuracy:

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')